# HW1 Binary Search using Inverted Index

In [34]:
import numpy as np
from gensim import corpora

In [35]:
dictionary = corpora.Dictionary.load('resources/vocab.dict')
doc_corpus = corpora.MmCorpus('resources/doc_corpus.mm')

In [125]:
# IDEA
# [
#   word0: [docID0, docID1, ... ,docIDN],
#   word1: [docID0, docID1, ... ,docIDN],
#   ... ,
#   wordM: [docID0, docID1, ... , docIDN]
# ]

# Creates the matrix
inverted_index = np.zeros((len(dictionary), len(doc_corpus)), dtype=np.bool_)

print("Calculating inverted index matrix!")

for term_indx in range(len(dictionary)):
    
    # Prints execution status
    print("Completion: {:4.2f} %".format((term_indx + 1) / len(dictionary) * 100), end="\r")
    
    for doc_indx in range(len(doc_corpus)):
        
        doc = doc_corpus[doc_indx]
        
        for token in doc:
            
            if term_indx == token[0]:
                inverted_index[term_indx, doc_indx] = True
                break
            
            else:
                inverted_index[term_indx, doc_indx] = False
                
np.save('./data/IImatrix.npy', inverted_index)

Calculating inverted index matrix!


In [253]:
def merge(pl0, pl1):

    merged = []

    finish = False
    pointers = np.zeros((2, 1), dtype=np.int_)

    while not finish:

        docs = np.zeros((2, 1), dtype=np.int_)

        try:
            docs[0] = pl0[pointers[0]] + 1
            docs[1] = pl1[pointers[1]] + 1
        except IndexError as e:
            finish = True

        if docs[0] == docs[1]:
            merged.append(docs[0])
            pointers[0] += 1
            pointers[1] += 1

        else:
            if docs[0] < docs[1]:
                pointers[0] += 1
            else:
                pointers[1] += 1

    return merged

In [281]:
def inverted_index_query(inverted_index, query):
        
    posting_lists = []
    
    for token_id in query:
        pl = np.where(inverted_index[token_id])[0] + 1
        posting_lists.append(pl)
        
    if len(posting_lists) > 1:
        
        lens = np.asarray([len(x) for x in posting_lists])
        pl0 = posting_lists.pop(lens.argsort()[0])
        
        lens = np.asarray([len(x) for x in posting_lists])
        pl1 = posting_lists.pop(lens.argsort()[0])      
        
        merged = merge(pl0, pl1)
        
        if len(posting_lists) > 1:
            pass
            
        else:
            merged = merge(merged, posting_lists[0])
        
        return merged
        
    else:
        return posting_lists[0]

In [282]:
query_corpus = corpora.MmCorpus("resources/query_corpus.mm")

In [283]:
for query in query_corpus:
    
    query_tokens = []
    
    for token in query:
        query_tokens.append(token[0])
        
    print(inverted_index_query(inverted_index, query_tokens))

TypeError: only integer scalar arrays can be converted to a scalar index